In [1]:
# %%
import sys, os
try:
    # ✅ Running from a Python script (.py file)
    TOOLS_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__)))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    TOOLS_PATH = os.path.abspath(os.path.join(os.getcwd()))

SRC_PATH = os.path.join(TOOLS_PATH)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

✅ SRC path added: /home/prashant-agrawal/projects/company_talk2data/src


In [21]:
import os
import openai
from langsmith import Client

LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT= "https://api.smith.langchain.com"
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY") 
openai.api_key=os.environ["OPENAI_API_KEY"]

# Set tracing in LangGraph
from langchain_core.tracers import ConsoleCallbackHandler
callbacks = [ConsoleCallbackHandler()]  #

In [ ]:
# src/main.py

from graph.workflow import build_graph

def run_once(user_query: str):
    """
    Build the graph, run it end‑to‑end, and return the final AgentState.
    """
    # 1) Compile the workflow
    graph = build_graph()

    # 2) Prepare the initial state for LangGraph
    #    All nodes share this same AgentState schema.
    initial_state = {
        "messages": [ {"role": "user", "content": user_query} ]
    }

    # 3) Invoke the graph synchronously (follows supervisor→enhancer→search→END)
    final_state = graph.invoke(initial_state)

    # 4) Inspect or return the final state
    return final_state

if __name__ == "__main__":
    query = "Find 5 B2B SaaS startups in India "
    result = run_once(query)

    print("=== Final State ===")
    print(f"Enhanced Query   : {result.get('enhanced_query')}")
    print(f"Filters          : {result.get('filters')}")
    print(f"Top‑K (k)        : {result.get('k')}")
    print(f"Results          : {result.get('retrieved_results')}")
    print(f"Trace Actions    : {result.get('actions')}")
    print(f"Trace Observations: {result.get('observations')}")

In [7]:
# main.py  ── run with:  python main.py "Find 5 B2B SaaS startups in India"
# -------------------------------------------------------------------------
from typing import Dict, Any
import sys, json

# 1️⃣  Import your pre-built agents + their tool lists
      
from agents.enhancer_agent import enhancer_agent
from agents.qdrant_search_agent import qdrant_search_agent
from tools.enhancer_tools_registry import enhancer_tools
from tools.qdrant_tools_registry import qdrant_tools
from langchain.agents import AgentExecutor

# -------------------------------------------------------------------------
# 2️⃣  Build executors (one line each)
enhancer_executor = AgentExecutor(
    agent=enhancer_agent,
    tools=enhancer_tools,
    verbose=False,
    handle_parsing_errors=True,
)

query = "Find 3 top startups in healthcare in India"

# 3️⃣  Run the enhancer agent
enhanced_query = enhancer_executor.invoke({"input": query})
print(f"Enhanced Query: {enhanced_query['output']}")

payload = enhanced_query["output"] 
# 4️⃣  Parse the enhanced query
query   = payload.get("query")
filters = payload.get("filters")
k       = payload.get("k")

print(f"Query: {query}")
print(f"Filters: {filters}")
print(f"Top-K (k): {k}")

Enhanced Query: {'query': 'List the top 3 startups in healthcare in India', 'filters': {'industry_sector': ['healthcare'], 'location': 'India', 'company_type': 'startup'}, 'k': 3}
Query: List the top 3 startups in healthcare in India
Filters: {'industry_sector': ['healthcare'], 'location': 'India', 'company_type': 'startup'}
Top-K (k): 3


In [8]:

# executor
qdrant_executor = AgentExecutor(
    agent                 = qdrant_search_agent,
    tools                 = qdrant_tools,   # list of tools
    verbose               = False,
    handle_parsing_errors = True,
    max_iterations        = 3
)

# call – flat arguments
result = qdrant_executor.invoke({
    "input":   query,        # plain string
    "filters": filters,      # dict or None
    "k":       k,            # int
})

print(json.dumps(result["output"], indent=2))  


[
  {
    "id": 97,
    "score": 0.37383556,
    "payload": {
      "company_name": "curefit",
      "legal_entity_type": "pvt ltd",
      "state": "west bengal",
      "headquarters_city": "bengaluru",
      "year_founded": "2022",
      "company_website": "https://durham-elliott.in",
      "logo_url": "https://logo.clearbit.com/durham-elliott.in",
      "company_description_short": "realigned secondary standardization",
      "company_description_long": "past off group positive. do total another past government.\r\nsouthern health southern rest couple. quickly resource foreign wait director remain establish necessary. sign real in like he far direction use.\r\nwatch we anyone. watch before buy scientist herself best use film. investment long know none.",
      "industry_sector": "healthtech",
      "total_funding_raised_inr": "\u20b9408 cr",
      "number_of_funding_rounds": "2",
      "latest_funding_round_type": "series c",
      "latest_funding_date": "2025-04-21",
      "lead_inv